In [4]:
!pip install imageio
!apt-get install -y ffmpeg

^C


In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

class SchwarzschildRayTracerBatch:
    def __init__(self, bh_mass=1.0, device=None):
        self.G = 1.0
        self.c = 1.0
        self.E = 1.0
        self.M = bh_mass
        self.rs = 2 * self.G * self.M / self.c**2
        self.b_max = 15 * self.rs
        self.device = device if device else (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

    def V(self, r, b):
        return (b**2 / (2 * r**2) - self.M * b**2 / r**3)

    def dV_dr(self, r, b):
        return (-b**2 / r**3 + 3 * self.M * b**2 / r**4)

    def system(self, y, b):
        r = y[:, 0]
        phi = y[:, 1]
        dr_dtau = y[:, 2]

        d2r_dtau2 = -self.dV_dr(r, b)
        dphi_dtau = b / r**2

        dydtau = torch.stack([dr_dtau, dphi_dtau, d2r_dtau2], dim=1)
        return dydtau

    def rkf45_step_batch(self, f, t, y, h, b):
        a = [0, 1/4, 3/8, 12/13, 1, 1/2]
        b_ = [
            [],
            [1/4],
            [3/32, 9/32],
            [1932/2197, -7200/2197, 7296/2197],
            [439/216, -8, 3680/513, -845/4104],
            [-8/27, 2, -3544/2565, 1859/4104, -11/40]
        ]
        c4 = [25/216, 0, 1408/2565, 2197/4104, -1/5, 0]
        c5 = [16/135, 0, 6656/12825, 28561/56430, -9/50, 2/55]

        k1 = f(y, b)
        k2 = f(y + h * (b_[1][0] * k1), b)
        k3 = f(y + h * (b_[2][0] * k1 + b_[2][1] * k2), b)
        k4 = f(y + h * (b_[3][0] * k1 + b_[3][1] * k2 + b_[3][2] * k3), b)
        k5 = f(y + h * (b_[4][0] * k1 + b_[4][1] * k2 + b_[4][2] * k3 + b_[4][3] * k4), b)
        k6 = f(y + h * (b_[5][0] * k1 + b_[5][1] * k2 + b_[5][2] * k3 + b_[5][3] * k4 + b_[5][4] * k5), b)

        y4 = y + h * (c4[0]*k1 + c4[2]*k3 + c4[3]*k4 + c4[4]*k5)
        y5 = y + h * (c5[0]*k1 + c5[2]*k3 + c5[3]*k4 + c5[4]*k5 + c5[5]*k6)

        error = torch.norm(y5 - y4, dim=1)
        return y5, error

    def solve_batch(self, b_array, r0=100.0, phi0=0.0, tau_span=(0, 140), h=0.1, tol=1e-5):
        b = torch.tensor(b_array, dtype=torch.float32, device=self.device)
        batch_size = b.shape[0]
        r0_t = torch.full((batch_size,), r0, dtype=torch.float32, device=self.device)
        phi0_t = torch.full((batch_size,), phi0, dtype=torch.float32, device=self.device)

        V0 = self.V(r0_t, b)
        dr0 = -torch.sqrt(torch.clamp(self.E**2 - 2 * V0, min=0.0))

        y = torch.stack([r0_t, phi0_t, dr0], dim=1)
        t = tau_span[0]
        tf = tau_span[1]

        ys = [y.cpu()]
        active = torch.ones(batch_size, dtype=torch.bool, device=self.device)

        while t < tf and active.any():
            y_next, error = self.rkf45_step_batch(self.system, t, y, h, b)
            y = torch.where(active.unsqueeze(1), y_next, y)
            active = active & (y[:, 0] > self.rs)
            ys.append(y.cpu())
            t += h

        return torch.stack(ys)

    def render(self, N_x=100, N_y=100, gamma_deg=20, save_path=None):
        ancho, alto = N_x, N_y
        imagen = Image.new('RGB', (ancho, alto), color=(0,0,0))
        pixeles = imagen.load()

        R= self.matriz_rotacion(0,0, gamma_deg)

        xs = np.linspace(-1, 1, ancho)
        ys = np.linspace(-1, 1, alto)
        xv, yv = np.meshgrid(xs, ys)

        b_vals = self.b_max * np.sqrt(xv.flatten()**2 + yv.flatten()**2)

        sol_batch = self.solve_batch(b_vals, r0=100*self.rs, tau_span=(0,2000), h=0.1)
        rs = self.rs

        r_vals = sol_batch[:, :, 0].numpy()
        phi_vals = sol_batch[:, :, 1].numpy()

        idx = 0
        # 1. Prepara coordenadas y trayectorias
        rs = self.rs
        theta_flat = torch.atan2(torch.tensor(yv, device=self.device), torch.tensor(xv, device=self.device)).flatten()  # (N_pix,)
        r_traj = torch.tensor(r_vals, device=self.device)   # (N_steps, N_pix)
        phi_traj = torch.tensor(phi_vals, device=self.device)  # (N_steps, N_pix)

        # 2. Expand para broadcasting
        theta_exp = theta_flat.unsqueeze(0).expand_as(r_traj)  # (N_steps, N_pix)

        # 3. Coordenadas esféricas a cartesianas
        x = torch.cos(theta_exp) * torch.sin(phi_traj)
        y = torch.sin(theta_exp) * torch.sin(phi_traj)
        z = torch.cos(phi_traj)

        # 4. Aplica matriz de rotación (en torch)
        R_torch = torch.tensor(self.matriz_rotacion(0, 0, gamma_deg), device=self.device)  # (3, 3)
        coords = torch.stack([x, y, z], dim=0)  # (3, N_steps, N_pix)
        coords_rot = torch.einsum('ij,jkl->ikl', R_torch, coords)  # (3, N_steps, N_pix)
        x1, y1, z1 = coords_rot[0], coords_rot[1], coords_rot[2]  # Cada uno (N_steps, N_pix)

        # 5. Evalúa condición del disco
        R2 = x1**2 + z1**2
        A = (R2 > 8) & (R2 < 17)
        B = (torch.abs(torch.atan(y1 / R2)) < 0.03)
        cond = A & B  # (N_steps, N_pix)

        # 6. Para cada rayo (columna), si hay algún punto True => pinta de naranja
        toca_disco = cond.any(dim=0).cpu().numpy().reshape(N_y, N_x)  # Mapa 2D

        # 7. Pinta imagen
        for i in range(N_x):
            for j in range(N_y):
                if toca_disco[j, i]:
                    pixeles[i, j] = (255, 128, 0)

        if save_path:
            imagen.save(save_path)
        else:
          image.save("Imagen_horizonte_sucesos.png")
          from google.colab import files
          files.download("Imagen_disco_de_acreción.png")

    def matriz_rotacion(self, theta_deg=0, phi_deg=0, gamma_deg=0):
        theta = np.deg2rad(theta_deg)
        phi = np.deg2rad(phi_deg)
        gamma= np.deg2rad(gamma_deg)
        Rz = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta),  np.cos(theta), 0],
            [0, 0, 1]
        ])
        Ry = np.array([
            [np.cos(phi), 0, np.sin(phi)],
            [0, 1, 0],
            [-np.sin(phi), 0, np.cos(phi)]
        ])
        Rx = np.array([
            [np.cos(gamma), 0, np.sin(gamma)],
            [0, 1, 0],
            [-np.sin(gamma), 0, np.cos(gamma)]
        ])
        return Rz @ Ry @ Rx

rt_batch = SchwarzschildRayTracerBatch(bh_mass=1.0)
rt_batch.render(N_x=100, N_y=100)



KeyboardInterrupt: 

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from scipy.interpolate import interp1d
from tqdm import tqdm

class SchwarzschildRayTracerBatch:
    def __init__(self, bh_mass=1.0, device=None):
        self.G = 1.0
        self.c = 1.0
        self.E = 1.0
        self.M = bh_mass
        self.rs = 2 * self.G * self.M / self.c**2
        self.b_max = 15 * self.rs
        self.device = device if device else (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

    def V(self, r, b):
        return (b**2 / (2 * r**2) - self.M * b**2 / r**3)

    def dV_dr(self, r, b):
        return (-b**2 / r**3 + 3 * self.M * b**2 / r**4)

    def system(self, y, b):
        r = y[:, 0]
        phi = y[:, 1]
        dr_dtau = y[:, 2]

        d2r_dtau2 = -self.dV_dr(r, b)
        dphi_dtau = b / r**2

        dydtau = torch.stack([dr_dtau, dphi_dtau, d2r_dtau2], dim=1)
        return dydtau

    def rkf45_step_batch(self, f, t, y, h, b):
        a = [0, 1/4, 3/8, 12/13, 1, 1/2]
        b_ = [
            [],
            [1/4],
            [3/32, 9/32],
            [1932/2197, -7200/2197, 7296/2197],
            [439/216, -8, 3680/513, -845/4104],
            [-8/27, 2, -3544/2565, 1859/4104, -11/40]
        ]
        c4 = [25/216, 0, 1408/2565, 2197/4104, -1/5, 0]
        c5 = [16/135, 0, 6656/12825, 28561/56430, -9/50, 2/55]

        k1 = f(y, b)
        k2 = f(y + h * (b_[1][0] * k1), b)
        k3 = f(y + h * (b_[2][0] * k1 + b_[2][1] * k2), b)
        k4 = f(y + h * (b_[3][0] * k1 + b_[3][1] * k2 + b_[3][2] * k3), b)
        k5 = f(y + h * (b_[4][0] * k1 + b_[4][1] * k2 + b_[4][2] * k3 + b_[4][3] * k4), b)
        k6 = f(y + h * (b_[5][0] * k1 + b_[5][1] * k2 + b_[5][2] * k3 + b_[5][3] * k4 + b_[5][4] * k5), b)

        y4 = y + h * (c4[0]*k1 + c4[2]*k3 + c4[3]*k4 + c4[4]*k5)
        y5 = y + h * (c5[0]*k1 + c5[2]*k3 + c5[3]*k4 + c5[4]*k5 + c5[5]*k6)

        error = torch.norm(y5 - y4, dim=1)
        return y5, error

    def solve_batch(self, b_array, r0=100.0, phi0=0.0, tau_span=(0, 140), h=0.1, tol=1e-5, N=1000):
        b = torch.tensor(b_array, dtype=torch.float32, device=self.device)
        batch_size = b.shape[0]
        r0_t = torch.full((batch_size,), r0, dtype=torch.float32, device=self.device)
        phi0_t = torch.full((batch_size,), phi0, dtype=torch.float32, device=self.device)

        V0 = self.V(r0_t, b)
        dr0 = -torch.sqrt(torch.clamp(self.E**2 - 2 * V0, min=0.0))

        y = torch.stack([r0_t, phi0_t, dr0], dim=1)
        t = tau_span[0]
        tf = tau_span[1]

        ys = [y.cpu()]
        active = torch.ones(batch_size, dtype=torch.bool, device=self.device)

        m=int((tf-t)/h/N)
        n=0

        with tqdm(total=N, desc="Renderizando", unit="px") as pbar:
          while t < tf and active.any():
            y_next, error = self.rkf45_step_batch(self.system, t, y, h, b)
            y = torch.where(active.unsqueeze(1), y_next, y)
            active = active & (y[:, 0] > self.rs)
            n +=1
            if n==m:
              ys.append(y.cpu())
              n=0
              pbar.update(1)
            t += h

        return torch.stack(ys)

    def render(self, N_x=100, N_y=100, gamma_deg=20, save_path=None):
        ancho, alto = N_x, N_y
        imagen = Image.new('RGB', (ancho, alto), color=(0,0,0))
        pixeles = imagen.load()

        R= self.matriz_rotacion(0,0, gamma_deg)

        xs = np.linspace(-1, 1, ancho)
        ys = np.linspace(-1, 1, alto)
        xv, yv = np.meshgrid(xs, ys)

        b_vals = self.b_max * np.sqrt(xv.flatten()**2 + yv.flatten()**2)
        N=400

        sol_batch = self.solve_batch(b_vals, r0=100*self.rs, tau_span=(0,2000), h=0.1, N=N)
        rs = self.rs

        r_vals = sol_batch[:, :, 0].numpy()
        phi_vals = sol_batch[:, :, 1].numpy()

        x1 =np.zeros(N)
        y1 =np.zeros(N)
        z1 =np.zeros(N)

        idx = 0
        with tqdm(total=ancho, desc="Renderizando", unit="px") as pbar:
          for i in range(ancho):
              for j in range(alto):
                  b = self.b_max * np.sqrt(xv[j,i]**2 + yv[j,i]**2)

                  r_traj = r_vals[:, idx]
                  phi_traj = phi_vals[:, idx]
                  #print(len(r_traj))

                  theta = np.arctan2(yv[j,i], xv[j,i])

                  x = r_traj*np.cos(theta) * np.sin(phi_traj)
                  y = r_traj*np.sin(theta) * np.sin(phi_traj)
                  z = r_traj*np.cos(phi_traj)

                  tau_original = np.linspace(0, len(r_traj)-1, len(r_traj))  # o el vector real de tau si tienes

                  r2 = np.sqrt(x**2 + y**2 + z**2)

                  indices_validos = np.where((r2 > 6) & (r2 < 20))[0]

                  if len(indices_validos) > 1:

                    tau_sub = tau_original[indices_validos]
                    r_sub = r_traj[indices_validos]
                    phi_sub = phi_traj[indices_validos]

                    if len(tau_sub) >= 4:  # mínimo 4 puntos para 'cubic'
                        interp_r = interp1d(tau_sub, r_sub, kind='cubic')
                        interp_phi = interp1d(tau_sub, phi_sub, kind='cubic')
                    else:
                        interp_r = interp1d(tau_sub, r_sub, kind='linear')
                        interp_phi = interp1d(tau_sub, phi_sub, kind='linear')

                    tau_fino = np.linspace(tau_sub[0], tau_sub[-1], num=300)

                    r_interp = interp_r(tau_fino)
                    phi_interp = interp_phi(tau_fino)

                    # Calcular coordenadas 3D interpoladas
                    x_interp = r_interp * np.cos(theta) * np.sin(phi_interp)
                    y_interp = r_interp * np.sin(theta) * np.sin(phi_interp)
                    z_interp = r_interp * np.cos(phi_interp)

                    points = np.vstack((x_interp, y_interp, z_interp)).T  # shape (N, 3)
                    rotated_points = points @ R.T  # shape (N, 3)

                    x1 = rotated_points[:, 0]
                    y1 = rotated_points[:, 1]
                    z1 = rotated_points[:, 2]

                    A = (np.sqrt(x1**2 + z1**2) > 8.62) & (np.sqrt(x1**2 + z1**2) < 18.32)
                    B = np.abs(np.arctan(y1 / np.sqrt(x1**2 + z1**2))) < 0.03

                    if np.any(A & B):
                        pixeles[i, j] = (255, 128, 0)

                    else:
                        pixeles[i, j] = (0, 0, 0)
                  else:
                    pixeles[i, j] = (0, 0, 0)

                  idx += 1
              pbar.update(1)

        if save_path:
            imagen.save(save_path)
        else:
          imagen.save("Imagen_disco_de_acreción.png")
          from google.colab import files
          files.download("Imagen_disco_de_acreción.png")

    def matriz_rotacion(self, theta_deg=0, phi_deg=0, gamma_deg=0):
        theta = np.deg2rad(theta_deg)
        phi = np.deg2rad(phi_deg)
        gamma= np.deg2rad(gamma_deg)
        Rz = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta),  np.cos(theta), 0],
            [0, 0, 1]
        ])
        Ry = np.array([
            [np.cos(phi), 0, np.sin(phi)],
            [0, 1, 0],
            [-np.sin(phi), 0, np.cos(phi)]
        ])
        Rx = np.array([
            [1, 0, 0],
            [0, np.cos(gamma), np.sin(gamma)],
            [0, -np.sin(gamma), np.cos(gamma)]
        ])
        return Rz @ Ry @ Rx

rt_batch = SchwarzschildRayTracerBatch(bh_mass=1.0)
rt_batch.render(N_x=700, N_y=700)

Renderizando:  63%|██████▎   | 251/400 [01:27<00:52,  2.86px/s]


KeyboardInterrupt: 

In [ ]:
import imageio
from IPython.display import FileLink
from tqdm import tqdm


# Crear directorio de salida
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

# Crear instancia
rt_batch = SchwarzschildRayTracerBatch(bh_mass=1.0)

# Ángulos de rotación
phis = np.arange(0, 360, 10)

# Renderizar
for phi in tqdm(phis, desc="Renderizando imágenes"):
    filename = os.path.join(output_dir, f"frame_{phi:03d}.png")
    rt_batch.render(N_x=300, N_y=300, phi_deg=phi, save_path=filename)

# Crear video
video_filename = "rotacion.mp4"
with imageio.get_writer(video_filename, fps=10) as writer:
    for phi in phis:
        filename = os.path.join(output_dir, f"frame_{phi:03d}.png")
        writer.append_data(imageio.imread(filename))

print("✅ Video creado:", video_filename)
FileLink(video_filename)


In [ ]:
from google.colab import files
files.download("rotacion.mp4")

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from scipy.interpolate import interp1d
from tqdm import tqdm
from google.colab import files
import imageio

class SchwarzschildRayTracerBatch:
    def __init__(self, bh_mass=1.0, device=None):
        self.G = 1.0
        self.c = 1.0
        self.E = 1.0
        self.M = bh_mass
        self.rs = 2 * self.G * self.M / self.c**2
        self.b_max = 15 * self.rs
        self.device = device if device else (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

    def V(self, r, b):
        return (b**2 / (2 * r**2) - self.M * b**2 / r**3)

    def dV_dr(self, r, b):
        return (-b**2 / r**3 + 3 * self.M * b**2 / r**4)

    def system(self, y, b):
        r = y[:, 0]
        phi = y[:, 1]
        dr_dtau = y[:, 2]

        d2r_dtau2 = -self.dV_dr(r, b)
        dphi_dtau = b / r**2

        dydtau = torch.stack([dr_dtau, dphi_dtau, d2r_dtau2], dim=1)
        return dydtau

    def rkf45_step_batch(self, f, t, y, h, b):
        a = [0, 1/4, 3/8, 12/13, 1, 1/2]
        b_ = [
            [],
            [1/4],
            [3/32, 9/32],
            [1932/2197, -7200/2197, 7296/2197],
            [439/216, -8, 3680/513, -845/4104],
            [-8/27, 2, -3544/2565, 1859/4104, -11/40]
        ]
        c4 = [25/216, 0, 1408/2565, 2197/4104, -1/5, 0]
        c5 = [16/135, 0, 6656/12825, 28561/56430, -9/50, 2/55]

        k1 = f(y, b)
        k2 = f(y + h * (b_[1][0] * k1), b)
        k3 = f(y + h * (b_[2][0] * k1 + b_[2][1] * k2), b)
        k4 = f(y + h * (b_[3][0] * k1 + b_[3][1] * k2 + b_[3][2] * k3), b)
        k5 = f(y + h * (b_[4][0] * k1 + b_[4][1] * k2 + b_[4][2] * k3 + b_[4][3] * k4), b)
        k6 = f(y + h * (b_[5][0] * k1 + b_[5][1] * k2 + b_[5][2] * k3 + b_[5][3] * k4 + b_[5][4] * k5), b)

        y4 = y + h * (c4[0]*k1 + c4[2]*k3 + c4[3]*k4 + c4[4]*k5)
        y5 = y + h * (c5[0]*k1 + c5[2]*k3 + c5[3]*k4 + c5[4]*k5 + c5[5]*k6)

        error = torch.norm(y5 - y4, dim=1)
        return y5, error

    def solve_batch(self, b_array, r0=100.0, phi0=0.0, tau_span=(0, 140), h=0.1, tol=1e-5, N=1000):
        b = torch.tensor(b_array, dtype=torch.float32, device=self.device)
        batch_size = b.shape[0]
        r0_t = torch.full((batch_size,), r0, dtype=torch.float32, device=self.device)
        phi0_t = torch.full((batch_size,), phi0, dtype=torch.float32, device=self.device)

        V0 = self.V(r0_t, b)
        dr0 = -torch.sqrt(torch.clamp(self.E**2 - 2 * V0, min=0.0))

        y = torch.stack([r0_t, phi0_t, dr0], dim=1)
        t = tau_span[0]
        tf = tau_span[1]

        ys = [y.cpu()]
        active = torch.ones(batch_size, dtype=torch.bool, device=self.device)

        m=int((tf-t)/h/N)
        n=0
        with tqdm(total=N, desc="Renderizando", unit="px") as pbar:
          while t < tf and active.any():
            #while error>
            y_next, error = self.rkf45_step_batch(self.system, t, y, h, b)

            y = torch.where(active.unsqueeze(1), y_next, y)
            active = active & (y[:, 0] > self.rs)
            n +=1
            if n==m:
              ys.append(y.cpu())
              n=0
              pbar.update(1)
            t += h

        return torch.stack(ys)

    def render(self, N_x=100, N_y=100, gamma_deg=20):
        ancho, alto = N_x, N_y
        imagen = Image.new('RGB', (ancho, alto), color=(0,0,0))
        imagen2 = Image.new('RGB', (ancho, alto), color=(0,0,0))
        pixeles = imagen.load()
        pixeles2= imagen2.load()

        R= self.matriz_rotacion(0,0, gamma_deg)

        xs = np.linspace(-1, 1, ancho)
        ys = np.linspace(-1, 1, alto)
        xv, yv = np.meshgrid(xs, ys)

        b_vals = self.b_max * np.sqrt(xv.flatten()**2 + yv.flatten()**2)
        N=400

        sol_batch = self.solve_batch(b_vals, r0=100*self.rs, tau_span=(0,2000), h=0.1, N=N)
        rs = self.rs

        r_vals = sol_batch[:, :, 0].numpy()
        phi_vals = sol_batch[:, :, 1].numpy()

        x1 =np.zeros(N)
        y1 =np.zeros(N)
        z1 =np.zeros(N)

        gamma_deg_vals=np.linspace(0,180, 36)
        with tqdm(total=ancho*len(gamma_deg_vals), desc="Renderizando", unit="px") as pbar:
          with imageio.get_writer("rotacion.mp4", fps=10) as writer:
            for gamma_deg in gamma_deg_vals:
              R= self.matriz_rotacion(0,0, gamma_deg)
              idx = 0
              for i in range(ancho):
                  for j in range(alto):
                      b = self.b_max * np.sqrt(xv[j,i]**2 + yv[j,i]**2)

                      r_traj = r_vals[:, idx]
                      phi_traj = phi_vals[:, idx]
                      #print(len(r_traj))

                      theta = np.arctan2(yv[j,i], xv[j,i])

                      if b<=3*np.sqrt(3)/2*rs:
                        phi = np.interp(rs, r_traj[-2:], phi_traj[-2:])
                        x = rs * np.cos(theta) * np.sin(phi)
                        y = rs * np.sin(theta) * np.sin(phi)
                        z = -rs * np.cos(phi)
                        x1,y1,z1 = np.dot([x,y,z], R)
                        theta1 = np.arccos(y1 / rs)+np.pi/2
                        phi1 = np.arctan2(z1, x1)+np.pi

                        if ((int(phi1 / (2*np.pi) * 12) % 2) == 0) and ((int(theta1 / np.pi * 7) % 2) == 0):
                          color = (0, 0, 255)
                        elif ((int(phi1 / (2*np.pi) * 12 + 1) % 2) == 0) and ((int(theta1 / np.pi * 7 + 1) % 2) == 0):
                          color = (0, 0, 255)
                        else:
                          color = (0, 0, 0)

                        pixeles[i, j] = color

                        if (b<=rs):
                          phi0=np.arcsin(b/self.rs)
                          x = rs * np.cos(theta) * np.sin(phi0)
                          y = rs * np.sin(theta) * np.sin(phi0)
                          z = -rs * np.cos(phi0)
                          x1,y1,z1 = np.dot([x,y,z], R)
                          theta1 = np.arccos(y1 / rs)+np.pi/2
                          phi1 = np.arctan2(z1, x1)+np.pi
                          if ((int(phi1 / (2*np.pi) * 12) % 2) == 0) and ((int(theta1 / np.pi * 7) % 2) == 0):
                              color2 = (255, 0, 0)
                          elif ((int(phi1 / (2*np.pi) * 12 + 1) % 2) == 0) and ((int(theta1 / np.pi * 7 + 1) % 2) == 0):
                              color2 = (255, 0, 0)
                          else:
                            color2 = (0, 0, 0)

                          pixeles2[i, j] = color2

                      x = r_traj*np.cos(theta) * np.sin(phi_traj)
                      y = r_traj*np.sin(theta) * np.sin(phi_traj)
                      z = r_traj*np.cos(phi_traj)

                      tau_original = np.linspace(0, len(r_traj)-1, len(r_traj))  # o el vector real de tau si tienes

                      r = np.sqrt(x**2 + y**2 + z**2)

                      indices_validos = np.where((r > 7) & (r < 20))[0]

                      if len(indices_validos) > 1:

                        tau_sub = tau_original[indices_validos]
                        r_sub = r_traj[indices_validos]
                        phi_sub = phi_traj[indices_validos]

                        if len(tau_sub) >= 4:  # mínimo 4 puntos para 'cubic'
                            interp_r = interp1d(tau_sub, r_sub, kind='cubic')
                            interp_phi = interp1d(tau_sub, phi_sub, kind='cubic')
                        else:
                            interp_r = interp1d(tau_sub, r_sub, kind='linear')
                            interp_phi = interp1d(tau_sub, phi_sub, kind='linear')

                        tau_fino = np.linspace(tau_sub[0], tau_sub[-1], num=300)

                        r_interp = interp_r(tau_fino)
                        phi_interp = interp_phi(tau_fino)

                        # Calcular coordenadas 3D interpoladas
                        x_interp = r_interp * np.cos(theta) * np.sin(phi_interp)
                        y_interp = r_interp * np.sin(theta) * np.sin(phi_interp)
                        z_interp = r_interp * np.cos(phi_interp)

                        points = np.vstack((x_interp, y_interp, z_interp)).T  # shape (N, 3)
                        rotated_points = points @ R.T  # shape (N, 3)

                        x1 = rotated_points[:, 0]
                        y1 = rotated_points[:, 1]
                        z1 = rotated_points[:, 2]

                        A = (np.sqrt(x1**2 + z1**2) > 8.62) & (np.sqrt(x1**2 + z1**2) < 18.314)
                        B = np.abs(np.arctan(y1 / np.sqrt(x1**2 + z1**2))) < 0.03

                        if np.any(A & B):
                            pixeles[i, j] = (255, 128, 0)

                        else:
                            pixeles[i, j] = (0, 0, 0)

                      idx += 1
                  pbar.update(1)
            writer.append_data(imageio.imread(imagen))
        files.download("rotacion.mp4")

    def matriz_rotacion(self, theta_deg=0, phi_deg=0, gamma_deg=0):
        theta = np.deg2rad(theta_deg)
        phi = np.deg2rad(phi_deg)
        gamma= np.deg2rad(gamma_deg)
        Rz = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta),  np.cos(theta), 0],
            [0, 0, 1]
        ])
        Ry = np.array([
            [np.cos(phi), 0, np.sin(phi)],
            [0, 1, 0],
            [-np.sin(phi), 0, np.cos(phi)]
        ])
        Rx = np.array([
            [1, 0, 0],
            [0, np.cos(gamma), np.sin(gamma)],
            [0, -np.sin(gamma), np.cos(gamma)]
        ])
        return Rz @ Ry @ Rx

rt_batch = SchwarzschildRayTracerBatch(bh_mass=1.0)
rt_batch.render(N_x=700, N_y=700)

Renderizando: 100%|██████████| 400/400 [02:13<00:00,  3.00px/s]
Renderizando: 4566px [15:19,  4.97px/s]


KeyboardInterrupt: 

In [ ]:
import cupy as cp
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from scipy.interpolate import interp1d
from tqdm import tqdm
from google.colab import files
import imageio

class SchwarzschildRayTracerBatch:
    def __init__(self, bh_mass=1.0, device=None):
        self.G = 1.0
        self.c = 1.0
        self.E = 1.0
        self.M = bh_mass
        self.rs = 2 * self.G * self.M / self.c**2
        self.b_max = 15 * self.rs
        self.device = device if device else (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

    def V(self, r, b):
        return (b**2 / (2 * r**2) - self.M * b**2 / r**3)

    def dV_dr(self, r, b):
        return (-b**2 / r**3 + 3 * self.M * b**2 / r**4)

    def system(self, y, b):
        r = y[:, 0]
        phi = y[:, 1]
        dr_dtau = y[:, 2]

        d2r_dtau2 = -self.dV_dr(r, b)
        dphi_dtau = b / r**2

        dydtau = torch.stack([dr_dtau, dphi_dtau, d2r_dtau2], dim=1)
        return dydtau

    def rkf45_step_batch(self, f, t, y, h, b):
        a = [0, 1/4, 3/8, 12/13, 1, 1/2]
        b_ = [
            [],
            [1/4],
            [3/32, 9/32],
            [1932/2197, -7200/2197, 7296/2197],
            [439/216, -8, 3680/513, -845/4104],
            [-8/27, 2, -3544/2565, 1859/4104, -11/40]
        ]
        c4 = [25/216, 0, 1408/2565, 2197/4104, -1/5, 0]
        c5 = [16/135, 0, 6656/12825, 28561/56430, -9/50, 2/55]

        k1 = f(y, b)
        k2 = f(y + h * (b_[1][0] * k1), b)
        k3 = f(y + h * (b_[2][0] * k1 + b_[2][1] * k2), b)
        k4 = f(y + h * (b_[3][0] * k1 + b_[3][1] * k2 + b_[3][2] * k3), b)
        k5 = f(y + h * (b_[4][0] * k1 + b_[4][1] * k2 + b_[4][2] * k3 + b_[4][3] * k4), b)
        k6 = f(y + h * (b_[5][0] * k1 + b_[5][1] * k2 + b_[5][2] * k3 + b_[5][3] * k4 + b_[5][4] * k5), b)

        y4 = y + h * (c4[0]*k1 + c4[2]*k3 + c4[3]*k4 + c4[4]*k5)
        y5 = y + h * (c5[0]*k1 + c5[2]*k3 + c5[3]*k4 + c5[4]*k5 + c5[5]*k6)

        error = torch.norm(y5 - y4, dim=1)
        return y5, error

    def solve_batch(self, b_array, r0=100.0, phi0=0.0, tau_span=(0, 140), h=0.1, tol=1e-5, N=1000):
        b = torch.tensor(b_array, dtype=torch.float32, device=self.device)
        batch_size = b.shape[0]
        r0_t = torch.full((batch_size,), r0, dtype=torch.float32, device=self.device)
        phi0_t = torch.full((batch_size,), phi0, dtype=torch.float32, device=self.device)

        V0 = self.V(r0_t, b)
        dr0 = -torch.sqrt(torch.clamp(self.E**2 - 2 * V0, min=0.0))

        y = torch.stack([r0_t, phi0_t, dr0], dim=1)
        t = tau_span[0]
        tf = tau_span[1]

        ys = [y.cpu()]
        active = torch.ones(batch_size, dtype=torch.bool, device=self.device)

        m=int((tf-t)/h/N)
        n=0
        with tqdm(total=N, desc="Renderizando", unit="px") as pbar:
          while t < tf and active.any():
            #while error>
            y_next, error = self.rkf45_step_batch(self.system, t, y, h, b)

            y = torch.where(active.unsqueeze(1), y_next, y)
            active = active & (y[:, 0] > self.rs)
            n +=1
            if n==m:
              ys.append(y.cpu())
              n=0
              pbar.update(1)
            t += h

        return torch.stack(ys)

    def render(self, N_x=100, N_y=100, gamma_deg=20):
        ancho, alto = N_x, N_y
        imagen = Image.new('RGB', (ancho, alto), color=(0,0,0))
        imagen2 = Image.new('RGB', (ancho, alto), color=(0,0,0))
        pixeles = imagen.load()
        pixeles2= imagen2.load()

        R= self.matriz_rotacion(0,0, gamma_deg)

        xs = np.linspace(-1, 1, ancho)
        ys = np.linspace(-1, 1, alto)
        xv, yv = np.meshgrid(xs, ys)

        b_vals = self.b_max * np.sqrt(xv.flatten()**2 + yv.flatten()**2)
        N=400

        sol_batch = self.solve_batch(b_vals, r0=100*self.rs, tau_span=(0,2000), h=0.1, N=N)
        rs = self.rs

        r_vals = sol_batch[:, :, 0].numpy()
        phi_vals = sol_batch[:, :, 1].numpy()

        x1 =np.zeros(N)
        y1 =np.zeros(N)
        z1 =np.zeros(N)

        gamma_deg_vals=np.linspace(0,180, 36)
        with tqdm(total=ancho*len(gamma_deg_vals), desc="Renderizando", unit="px") as pbar:
          with imageio.get_writer("rotacion.mp4", fps=10) as writer:
            for gamma_deg in gamma_deg_vals:
              R= self.matriz_rotacion(0,0, gamma_deg)

              xv = cp.asarray(xv)
              yv = cp.asarray(yv)
              # Crear índices de grilla
              j_idx, i_idx = cp.meshgrid(cp.arange(alto), cp.arange(ancho), indexing='ij')
              flat_idx = j_idx * ancho + i_idx

              # Coordenadas
              x = xv[j_idx, i_idx]
              y = yv[j_idx, i_idx]
              theta = cp.arctan2(y, x)
              b = self.b_max * cp.sqrt(x**2 + y**2)

              # Máscaras
              mask1 = b <= (3 * cp.sqrt(3) / 2 * rs)
              mask2 = b <= rs

              # Salidas temporales
              color_img = cp.zeros((alto, ancho, 3), dtype=cp.uint8)
              color_img2 = cp.zeros((alto, ancho, 3), dtype=cp.uint8)

              # Procesamiento por píxel donde b <= 3√3/2·rs
              indices = cp.argwhere(mask1)


              for idx_pair in indices:
                  j, i = int(idx_pair[0]), int(idx_pair[1])
                  idx = j * ancho + i

                  r_traj = r_vals[:, idx]
                  phi_traj = phi_vals[:, idx]
                  th = float(theta[j, i])

                  # --- Interpolación (sigue en CPU por ahora)
                  phi = float(np.interp(rs, r_traj[-2:], phi_traj[-2:]))

                  x0 = rs * cp.cos(th) * cp.sin(phi)
                  y0 = rs * cp.sin(th) * cp.sin(phi)
                  z0 = -rs * cp.cos(phi)

                  x1, y1, z1 = cp.dot(cp.array([x0, y0, z0]), R)

                  theta1 = cp.arccos(y1 / rs) + cp.pi / 2
                  phi1 = cp.arctan2(z1, x1) + cp.pi

                  cond = ((int(phi1 / (2 * cp.pi) * 12) % 2 == 0) and
                          (int(theta1 / cp.pi * 7) % 2 == 0)) or \
                        ((int(phi1 / (2 * cp.pi) * 12 + 1) % 2 == 0) and
                          (int(theta1 / cp.pi * 7 + 1) % 2 == 0))

                  color_img[j, i] = cp.array([0, 0, 255]) if cond else cp.array([0, 0, 0])

                  # Interior (b <= rs)
                  if mask2[j, i]:
                      phi0 = cp.arcsin(b[j, i] / rs)
                      x0 = rs * cp.cos(th) * cp.sin(phi0)
                      y0 = rs * cp.sin(th) * cp.sin(phi0)
                      z0 = -rs * cp.cos(phi0)

                      x1, y1, z1 = cp.dot(cp.array([x0, y0, z0]), R)

                      theta1 = cp.arccos(y1 / rs) + cp.pi / 2
                      phi1 = cp.arctan2(z1, x1) + cp.pi

                      cond2 = ((int(phi1 / (2 * cp.pi) * 12) % 2 == 0) and
                              (int(theta1 / cp.pi * 7) % 2 == 0)) or \
                              ((int(phi1 / (2 * cp.pi) * 12 + 1) % 2 == 0) and
                              (int(theta1 / cp.pi * 7 + 1) % 2 == 0))

                      color_img2[j, i] = cp.array([255, 0, 0]) if cond2 else cp.array([0, 0, 0])

                  # Trayectoria completa (uso CPU para interp por ahora)
                  r_vec = r_traj
                  phi_vec = phi_traj
                  r_vec = cp.asarray(r_vec)
                  phi_vec = cp.asarray(phi_vec)

                  tau = np.linspace(0, len(r_vec) - 1, len(r_vec))
                  x_traj = r_vec * cp.cos(th) * cp.sin(phi_vec)
                  y_traj = r_vec * cp.sin(th) * cp.sin(phi_vec)
                  z_traj = r_vec * cp.cos(phi_vec)

                  r_mag = cp.sqrt(x_traj**2 + y_traj**2 + z_traj**2)
                  valid = cp.where((r_mag > 7) & (r_mag < 20))[0]

                  if len(valid) > 1:
                      tau_sub = tau[valid.get()]
                      r_sub = r_vec[valid]
                      phi_sub = phi_vec[valid]

                      if len(tau_sub) >= 4:
                          interp_r = interp1d(tau_sub, cp.asnumpy(r_sub), kind='cubic')
                          interp_phi = interp1d(tau_sub, cp.asnumpy(phi_sub), kind='cubic')
                      else:
                          interp_r = interp1d(tau_sub, cp.asnumpy(r_sub), kind='linear')
                          interp_phi = interp1d(tau_sub, cp.asnumpy(phi_sub), kind='linear')

                      tau_fine = np.linspace(tau_sub[0], tau_sub[-1], num=300)

                      r_interp = interp_r(tau_fine)
                      phi_interp = interp_phi(tau_fine)

                      x_interp = r_interp * np.cos(th) * np.sin(phi_interp)
                      y_interp = r_interp * np.sin(th) * np.sin(phi_interp)
                      z_interp = r_interp * np.cos(phi_interp)

                      points = np.vstack((x_interp, y_interp, z_interp))
                      points = cp.asarray(points)
                      rotated = R @ points

                      x1, y1, z1 = rotated

                      A = (np.sqrt(x1**2 + z1**2) > 8.62) & (np.sqrt(x1**2 + z1**2) < 18.314)
                      B = np.abs(np.arctan(y1 / np.sqrt(x1**2 + z1**2))) < 0.03

                      if np.any(A & B):
                          color_img[j, i] = cp.array([255, 128, 0])
                      else:
                          color_img[j, i] = cp.array([0, 0, 0])
              img_cpu = color_img.get()
              writer.append_data(imageio.imwrite('frame.png', img_cpu))
        files.download("rotacion.mp4")

    def matriz_rotacion(self, theta_deg=0, phi_deg=0, gamma_deg=0):
        theta = np.deg2rad(theta_deg)
        phi = np.deg2rad(phi_deg)
        gamma= np.deg2rad(gamma_deg)
        Rz = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta),  np.cos(theta), 0],
            [0, 0, 1]
        ])
        Ry = np.array([
            [np.cos(phi), 0, np.sin(phi)],
            [0, 1, 0],
            [-np.sin(phi), 0, np.cos(phi)]
        ])
        Rx = np.array([
            [1, 0, 0],
            [0, np.cos(gamma), np.sin(gamma)],
            [0, -np.sin(gamma), np.cos(gamma)]
        ])
        return cp.asarray(Rz @ Ry @ Rx)

rt_batch = SchwarzschildRayTracerBatch(bh_mass=1.0)
rt_batch.render(N_x=100, N_y=100)



Renderizando:  85%|████████▌ | 341/400 [00:36<00:06,  9.39px/s]